# HW5 Codes

In [3]:
%pip install --quiet pandas numpy altair vega_datasets

import pandas as pd
import numpy as np
import altair as alt

Note: you may need to restart the kernel to use updated packages.


### Load the data

In [4]:
illinois_url = 'https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv'

In [5]:
illinois = pd.read_csv(illinois_url)
#illinois.columns
#illinois.isna().mean().round(3) * 100

### Plot 1: License Type Distribution

  - What is being visualized: 
    - Create a **horizontal bar** plot describing the **distribution of license types by license status**. This allows us to see which license types are most common and how their statuses vary. I found `Detective Board` to be the most frequent license type. And more interestingly, `Not renewd` status is the second most common status across various license types after `Active`.
  - Encoding type discussion: 
    - X-axis: **Count of Licenses (Quantitative)** to provide horizontal plot. More infromation on how it is transformed below.
    - Y-axis: **License Type (Nominal)** sorted by count to highlight the most common license types.
    - Color: **License Status (Nominal)** to differentiate between different statuses of licenses.
    - Tooltip: license type, license status, and count information are provided for detailed understanding.
  - Color mapping: 
    - Different colors used to show **varying license statuses** (e.g., active, inactive, cancelled) for better visual distinction.
  - Any data transformations: 
    - Grouped the data by `License Type` and `License Status` to count the number of occurrences.
    - Applied with a **symlog scale** to accommodate wide and disproportional range of counts. As below 500 counts were very common across license types, while some license types had counts in the thousands. The symlog scale helps to visualize both small and large counts effectively.



In [6]:
Licence_counts = (
    illinois
    .groupby(['License Type', 'License Status'], dropna=False)
    .size()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)


chart1 = alt.Chart(Licence_counts).mark_bar().encode(
          x = alt.X('Count:Q',
                  title = 'Number of Licenses (symlog)',
                  scale = alt.Scale(type = 'symlog', constant=800)),
          y = alt.Y('License Type:N',
                  sort = '-x',
                  title = 'License Type'),
          color = alt.Color('License Status:N', title = 'License Status?'),
          tooltip = [alt.Tooltip('License Type:N', title = 'License Type'),
                     alt.Tooltip('License Status:N', title = 'License Status?'),
                     alt.Tooltip('Count:Q', title = 'Count')]
      ).properties(width=700, height=400, title='License Type Distribution').interactive()

chart1

alt.Chart(...)

### Plot 2: License Type Distribution by License Status per State

  - What is being visualized: 
    - Create a **heatmap** plot describing the distribution of license status by action applied per state. This allows us to see how different actions are associated with various license statuses across states. For example, in Illinois, I could find `No Action` × `NOT RENEWED` dominates — most cases look like non-disciplinary lapses (just not renewed). Also, serious actions don’t coexist with `ACTIVE` — Termination/Summary Suspension/Revocation rarely appear under ACTIVE status.
  - Encoding type discussion: 
    - X-axis: **License Status (Nominal)** to provide heatmap.
    - Y-axis: **Action (Nominal)** sorted by count to highlight the most common actions in a descending order.
    - Color: **Count of Licenses (Quantitative)** to differentiate between different counts of licenses for each action and license status combination.
    - Tooltip: license status, action, and count information are provided for detailed understanding.
  - Color mapping: 
    - Different colors (`scheme='cividis'`) used to show **varying degree of counts** (the more, the lighter) for better visual distinction.
  - Interactivity: 
    - A **dropdown menu** to select specific `States` is addeded. This allows us to filter the data by states and provide how the relation between action and license status differ across country. For example, I found serious disciplines in Action were present in IL, while they were very rare or absent in TX.
  - Any data transformations: 
    - Grouped the data by 'State', 'Action', and 'License Status' to count the number of occurrences.
    - Applied with a **log scale** on Y axis to accommodate disproportional range of counts. Some action-license status combinations had very low counts, while others had counts in the hundreds. The log-transformation helped to visualize both small and large counts effectively.


In [8]:

alt.data_transformers.enable("default", max_rows=None)  # error came so I added this line

#illinois['Action'].unique()

# Mutate NaNs in 'Action' to 'No Action' to show them in the plots
illinois2 = illinois.copy()
illinois2['Action2'] = (
    illinois2['Action'].astype('string').str.strip()
        .replace({'undefined': np.nan})
        .fillna('No Action')
)


## Data manipulation
# I got help from ChatGPT to create this heatmap code and calculation of total counts below.
# Create new dataset for heatmap  
illinois2 = (
    illinois2
      .groupby(['State', 'Action2', 'License Status'], dropna=False)
      .size()
      .reset_index(name='count')
)

# Calculate total counts of Action by State
illinois2['total'] = (
    illinois2
      .groupby(['State', 'Action2'])['count']
      .transform('sum')
)


## Drawing
# Interactivity added
states_sorted = sorted(illinois2['State'].dropna().unique().tolist())

state_sel = alt.selection_point(
    fields=['State'],
    bind=alt.binding_select(options=states_sorted, name='State: '),
    value=('IL' if 'IL' in states_sorted else states_sorted[0])   # start with Illinois
)

# Draw the second plot
chart2 = (
    alt.Chart(illinois2).mark_rect()
      .encode(
          x = alt.X('License Status:N', title = 'License Status', axis = alt.Axis(labelAngle=-25)),
          y = alt.Y('Action2:N', title='Action',
                  sort = alt.SortField(field = 'total', order = 'descending')), 
          color = alt.Color('count:Q', title = 'Count', scale = alt.Scale(scheme='cividis', type='log')),
          tooltip = ['State:N', 'Action2:N', 'License Status:N', alt.Tooltip('count:Q', title = 'Count')]
      ).add_params(state_sel)
      .transform_filter(state_sel)
      .properties(width=700, height=300,
                  title='Action x License Status filtered by State')
)

chart2

alt.Chart(...)

In [9]:
myJekyllDir = '/Users/Haki/Documents/GitHub/hopeintheair.github.io/assets/json/'
chart1.save(myJekyllDir + 'chart1.json')
chart2.save(myJekyllDir + 'chart2.json')

In [ ]:
#illinois.to_csv("/Users/Haki/Documents/GitHub/hopeintheair.github.io/assets/csv/licenses_fall2022.csv", index=True)